In [41]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, Imputer, LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.grid_search import RandomizedSearchCV, GridSearchCV
from sklearn.svm import SVC, LinearSVC
from xgboost import XGBClassifier
import lightgbm
import sys
sys.path.insert(0,'../scripts/')
from data_loader import load_data, extract_X_y, splitting
from sklearn_pandas import DataFrameMapper
from pipeline_utils import AddTwoCategoricalVariables, ColumnExtractor, DropColumns, FeatureNormalizer, FillNa, Numerical2Categorical, SexBinarizer, OneHotEncoding, FillNa2, Trial
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_path = '../data/train.csv'
train_path_pkl = '../data/train.pkl'
test_path = '../data/test.csv'
test_path_pkl = '../data/test.pkl'

In [3]:
titanic_data = load_data(train_path, train_path_pkl)

INFO:data_loader:
Timings: 'load_data'  10.57 ms


In [4]:
test_data = load_data(test_path, test_path_pkl)

INFO:data_loader:
Timings: 'load_data'  2.03 ms


### Splitting

In [5]:
X, y = extract_X_y(titanic_data)

INFO:data_loader:
Timings: 'extract_X_y'  3.29 ms


In [6]:
X_train, X_val, y_train, y_val = splitting(X,y)

INFO:data_loader:
Timings: 'splitting'  5.10 ms


## 1. XGBoost

In [7]:
age_range = [0, 15, 35, 50, 80]
age_label = [0, 1, 2, 3]

mapper = DataFrameMapper([
("Fare", [FillNa2("mean"), MinMaxScaler()], {'alias': 'Fare'}),
("Pclass", [FillNa2("mean"), MinMaxScaler()], {'alias': 'Pclass'}),
("Sex", FillNa2("mean"),  {'alias': "Sex"}),
("Age", FillNa2("mean"),  {'alias': "Age"}),
("SibSp", [FillNa2("mean"), MinMaxScaler()], {'alias': 'SibSp'}),
("Parch", [FillNa2("mean"), MinMaxScaler()], {'alias': 'Parch'}),
], df_out=True)
pipeline = Pipeline([
        ("mapper", mapper),
        ("sex_binarizer", SexBinarizer()),
        ("num2cat", Numerical2Categorical("Age", age_range, age_label)),
        ("add_age_sex", AddTwoCategoricalVariables("Age_cat", "Sex")),
        ("add_sex_class", AddTwoCategoricalVariables("Sex", "Pclass")),
        ("add_age_sex_class", AddTwoCategoricalVariables("Age_cat_Sex", "Pclass")),
        ("one_hot_encoding", OneHotEncoding(["Age_cat_Sex", "Sex_Pclass"])),
        ("drop_columns", DropColumns(["Age_cat"])),
        ('xgb', XGBClassifier())
])

In [8]:
pipeline.fit(X_train, y_train)
y_predict = pipeline.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_predict)

0.8857786357786358

## GridSearchCV

In [9]:
param_grid = {
    'xgb__max_depth': [2, 4, 5, 6, 7],
    'xgb__n_estimators': [150, 200, 250, 300, 350],
    'xgb__learning_rate': [0.01, 0.1, 0.2, 0.25],
    'xgb__subsample': [0.5, 0.75, 1],
    'xgb__colsample_bytree': [0.4, 0.6, 0.75, 0.9]
}

grid_xg = GridSearchCV(pipeline, cv=3, n_jobs=-1, param_grid=param_grid, scoring="roc_auc", verbose=1)
grid_xg.fit(X_train,y_train)

Fitting 3 folds for each of 1200 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:   39.4s
[Parallel(n_jobs=-1)]: Done 1576 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 2476 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 3576 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed:  3.4min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('mapper', DataFrameMapper(default=False, df_out=True,
        features=[('Fare', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Fare'}), ('Pclass', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Pclass'}), ('Sex', Fill...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'xgb__max_depth': [2, 4, 5, 6, 7], 'xgb__n_estimators': [150, 200, 250, 300, 350], 'xgb__learning_rate': [0.01, 0.1, 0.2, 0.25], 'xgb__subsample': [0.5, 0.75, 1], 'xgb__colsample_bytree': [0.4, 0.6, 0.75, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=1)

In [10]:
grid_xg.best_score_

0.8612598182388929

In [11]:
grid_xg.best_params_

{'xgb__colsample_bytree': 0.9,
 'xgb__learning_rate': 0.1,
 'xgb__max_depth': 2,
 'xgb__n_estimators': 250,
 'xgb__subsample': 0.5}

## RandomizedSearchCV

In [38]:
random_xg = RandomizedSearchCV(pipeline, cv=3, n_jobs=-1, param_distributions=param_grid, scoring="roc_auc", verbose=1)
random_xg.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.2s finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('mapper', DataFrameMapper(default=False, df_out=True,
        features=[('Fare', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Fare'}), ('Pclass', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Pclass'}), ('Sex', Fill...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params={}, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'randomforestclassifier__max_features': [0.25, 0.5, 0.75, 'auto'], 'randomforestclassifier__n_estimators': [60, 70, 80, 100]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='roc_auc', verbose=1)

In [40]:
random_xg.best_score_

0.8497859730074727

## 2. RandomForestClassifier

In [29]:
age_range = [0, 15, 35, 50, 80]
age_label = [0, 1, 2, 3]

mapper = DataFrameMapper([
("Fare", [FillNa2("mean"), MinMaxScaler()], {'alias': 'Fare'}),
("Pclass", [FillNa2("mean"), MinMaxScaler()], {'alias': 'Pclass'}),
("Sex", FillNa2("mean"),  {'alias': "Sex"}),
("Age", FillNa2("mean"),  {'alias': "Age"}),
("SibSp", [FillNa2("mean"), MinMaxScaler()], {'alias': 'SibSp'}),
("Parch", [FillNa2("mean"), MinMaxScaler()], {'alias': 'Parch'}),
], df_out=True)
pipeline = Pipeline([
#             ("features", FeatureUnion([]))
        ("mapper", mapper),
        ("sex_binarizer", SexBinarizer()),
        ("num2cat", Numerical2Categorical("Age", age_range, age_label)),
        ("add_age_sex", AddTwoCategoricalVariables("Age_cat", "Sex")),
        ("add_sex_class", AddTwoCategoricalVariables("Sex", "Pclass")),
        ("add_age_sex_class", AddTwoCategoricalVariables("Age_cat_Sex", "Pclass")),
        ("one_hot_encoding", OneHotEncoding(["Age_cat_Sex", "Sex_Pclass"])),
        ("drop_columns", DropColumns(["Age_cat"])),
        ('randomforestclassifier', RandomForestClassifier())
])

## Without Hyperparameter tuning

In [30]:
pipeline.fit(X_train, y_train)

y_predict = pipeline.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_predict)

0.8643500643500643

## GridSearchCV

In [31]:
param_grid = {
    'randomforestclassifier__max_features': [0.25, 0.5, 0.75, 'auto'],
    'randomforestclassifier__n_estimators': [60, 70, 80, 100],
# #     'randomforestclassifier__min_samples_split' : [2, 5, 10],
#     'randomforestclassifier__min_samples_leaf' : [1, 2, 4]
}

grid_rf = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, scoring="roc_auc", verbose=1)
grid_rf.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    7.0s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('mapper', DataFrameMapper(default=False, df_out=True,
        features=[('Fare', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Fare'}), ('Pclass', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Pclass'}), ('Sex', Fill...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'randomforestclassifier__max_features': [0.25, 0.5, 0.75, 'auto'], 'randomforestclassifier__n_estimators': [60, 70, 80, 100]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=1)

In [32]:
grid_rf.best_score_

0.8495040505092897

In [33]:
pipeline.named_steps['randomforestclassifier']

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [34]:
grid_rf.best_params_

{'randomforestclassifier__max_features': 'auto',
 'randomforestclassifier__n_estimators': 70}

## RandomizedSearchCV

In [35]:
random_rf = RandomizedSearchCV(pipeline, cv=3, n_jobs=1, param_distributions=param_grid, scoring="roc_auc", verbose=1)
random_rf.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    4.4s finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('mapper', DataFrameMapper(default=False, df_out=True,
        features=[('Fare', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Fare'}), ('Pclass', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Pclass'}), ('Sex', Fill...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'randomforestclassifier__max_features': [0.25, 0.5, 0.75, 'auto'], 'randomforestclassifier__n_estimators': [60, 70, 80, 100]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='roc_auc', verbose=1)

In [36]:
random_rf.best_score_

0.8521344895425661

In [37]:
random_rf.best_params_

{'randomforestclassifier__n_estimators': 60,
 'randomforestclassifier__max_features': 0.5}

## 3. CatBoost

In [44]:
age_range = [0, 15, 35, 50, 80]
age_label = [0, 1, 2, 3]

mapper = DataFrameMapper([
("Fare", [FillNa2("mean"), MinMaxScaler()], {'alias': 'Fare'}),
("Pclass", [FillNa2("mean"), MinMaxScaler()], {'alias': 'Pclass'}),
("Sex", FillNa2("mean"),  {'alias': "Sex"}),
("Age", FillNa2("mean"),  {'alias': "Age"}),
("SibSp", [FillNa2("mean"), MinMaxScaler()], {'alias': 'SibSp'}),
("Parch", [FillNa2("mean"), MinMaxScaler()], {'alias': 'Parch'}),
], df_out=True)
pipeline = Pipeline([
        ("mapper", mapper),
        ("sex_binarizer", SexBinarizer()),
        ("num2cat", Numerical2Categorical("Age", age_range, age_label)),
        ("add_age_sex", AddTwoCategoricalVariables("Age_cat", "Sex")),
        ("add_sex_class", AddTwoCategoricalVariables("Sex", "Pclass")),
        ("add_age_sex_class", AddTwoCategoricalVariables("Age_cat_Sex", "Pclass")),
        ("one_hot_encoding", OneHotEncoding(["Age_cat_Sex", "Sex_Pclass"])),
        ("drop_columns", DropColumns(["Age_cat"])),
        ('catboost', CatBoostClassifier())
])

In [45]:
pipeline.fit(X_train, y_train)

y_predict = pipeline.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_predict)

Learning rate set to 0.015219
0:	learn: 0.6831290	total: 89.6ms	remaining: 1m 29s
1:	learn: 0.6731371	total: 94.4ms	remaining: 47.1s
2:	learn: 0.6642967	total: 101ms	remaining: 33.5s
3:	learn: 0.6560082	total: 105ms	remaining: 26.3s
4:	learn: 0.6481142	total: 112ms	remaining: 22.3s
5:	learn: 0.6405101	total: 117ms	remaining: 19.3s
6:	learn: 0.6332307	total: 123ms	remaining: 17.5s
7:	learn: 0.6243438	total: 130ms	remaining: 16.1s
8:	learn: 0.6166579	total: 136ms	remaining: 15s
9:	learn: 0.6104643	total: 143ms	remaining: 14.2s
10:	learn: 0.6034315	total: 150ms	remaining: 13.4s
11:	learn: 0.5966365	total: 156ms	remaining: 12.8s
12:	learn: 0.5929537	total: 160ms	remaining: 12.1s
13:	learn: 0.5864198	total: 165ms	remaining: 11.6s
14:	learn: 0.5810618	total: 171ms	remaining: 11.2s
15:	learn: 0.5752134	total: 177ms	remaining: 10.9s
16:	learn: 0.5690964	total: 184ms	remaining: 10.6s
17:	learn: 0.5634384	total: 190ms	remaining: 10.4s
18:	learn: 0.5580126	total: 196ms	remaining: 10.1s
19:	learn:

162:	learn: 0.3847490	total: 1.1s	remaining: 5.68s
163:	learn: 0.3843013	total: 1.11s	remaining: 5.67s
164:	learn: 0.3841691	total: 1.12s	remaining: 5.67s
165:	learn: 0.3839463	total: 1.13s	remaining: 5.68s
166:	learn: 0.3837399	total: 1.14s	remaining: 5.68s
167:	learn: 0.3835671	total: 1.15s	remaining: 5.67s
168:	learn: 0.3832934	total: 1.15s	remaining: 5.67s
169:	learn: 0.3831020	total: 1.16s	remaining: 5.66s
170:	learn: 0.3828604	total: 1.17s	remaining: 5.66s
171:	learn: 0.3825797	total: 1.18s	remaining: 5.66s
172:	learn: 0.3824594	total: 1.18s	remaining: 5.66s
173:	learn: 0.3820878	total: 1.19s	remaining: 5.65s
174:	learn: 0.3819779	total: 1.2s	remaining: 5.64s
175:	learn: 0.3817509	total: 1.2s	remaining: 5.64s
176:	learn: 0.3816150	total: 1.21s	remaining: 5.63s
177:	learn: 0.3814139	total: 1.22s	remaining: 5.62s
178:	learn: 0.3811470	total: 1.22s	remaining: 5.6s
179:	learn: 0.3808892	total: 1.23s	remaining: 5.59s
180:	learn: 0.3806538	total: 1.23s	remaining: 5.58s
181:	learn: 0.38

347:	learn: 0.3532080	total: 2.31s	remaining: 4.33s
348:	learn: 0.3531304	total: 2.32s	remaining: 4.32s
349:	learn: 0.3531047	total: 2.32s	remaining: 4.31s
350:	learn: 0.3530661	total: 2.33s	remaining: 4.3s
351:	learn: 0.3530270	total: 2.33s	remaining: 4.29s
352:	learn: 0.3530267	total: 2.34s	remaining: 4.29s
353:	learn: 0.3529262	total: 2.35s	remaining: 4.28s
354:	learn: 0.3528771	total: 2.35s	remaining: 4.27s
355:	learn: 0.3526809	total: 2.36s	remaining: 4.26s
356:	learn: 0.3524674	total: 2.36s	remaining: 4.26s
357:	learn: 0.3524096	total: 2.37s	remaining: 4.25s
358:	learn: 0.3521446	total: 2.38s	remaining: 4.25s
359:	learn: 0.3520557	total: 2.38s	remaining: 4.24s
360:	learn: 0.3519513	total: 2.39s	remaining: 4.23s
361:	learn: 0.3518348	total: 2.39s	remaining: 4.22s
362:	learn: 0.3516134	total: 2.4s	remaining: 4.21s
363:	learn: 0.3515427	total: 2.41s	remaining: 4.21s
364:	learn: 0.3513885	total: 2.42s	remaining: 4.2s
365:	learn: 0.3513496	total: 2.42s	remaining: 4.2s
366:	learn: 0.35

513:	learn: 0.3276244	total: 3.5s	remaining: 3.31s
514:	learn: 0.3274347	total: 3.51s	remaining: 3.3s
515:	learn: 0.3272683	total: 3.52s	remaining: 3.3s
516:	learn: 0.3271871	total: 3.52s	remaining: 3.29s
517:	learn: 0.3270837	total: 3.53s	remaining: 3.29s
518:	learn: 0.3269238	total: 3.54s	remaining: 3.28s
519:	learn: 0.3264710	total: 3.54s	remaining: 3.27s
520:	learn: 0.3263770	total: 3.55s	remaining: 3.27s
521:	learn: 0.3261561	total: 3.56s	remaining: 3.26s
522:	learn: 0.3256338	total: 3.56s	remaining: 3.25s
523:	learn: 0.3254574	total: 3.57s	remaining: 3.24s
524:	learn: 0.3253276	total: 3.58s	remaining: 3.23s
525:	learn: 0.3253274	total: 3.58s	remaining: 3.23s
526:	learn: 0.3251703	total: 3.59s	remaining: 3.22s
527:	learn: 0.3251677	total: 3.59s	remaining: 3.21s
528:	learn: 0.3251675	total: 3.59s	remaining: 3.2s
529:	learn: 0.3250104	total: 3.6s	remaining: 3.19s
530:	learn: 0.3248940	total: 3.61s	remaining: 3.19s
531:	learn: 0.3247824	total: 3.61s	remaining: 3.18s
532:	learn: 0.324

680:	learn: 0.3021730	total: 4.68s	remaining: 2.19s
681:	learn: 0.3020006	total: 4.69s	remaining: 2.19s
682:	learn: 0.3018370	total: 4.7s	remaining: 2.18s
683:	learn: 0.3017996	total: 4.71s	remaining: 2.17s
684:	learn: 0.3017012	total: 4.71s	remaining: 2.17s
685:	learn: 0.3014378	total: 4.72s	remaining: 2.16s
686:	learn: 0.3013227	total: 4.73s	remaining: 2.15s
687:	learn: 0.3013212	total: 4.73s	remaining: 2.15s
688:	learn: 0.3011647	total: 4.74s	remaining: 2.14s
689:	learn: 0.3011632	total: 4.74s	remaining: 2.13s
690:	learn: 0.3010703	total: 4.75s	remaining: 2.12s
691:	learn: 0.3009561	total: 4.75s	remaining: 2.12s
692:	learn: 0.3008787	total: 4.76s	remaining: 2.11s
693:	learn: 0.3007076	total: 4.76s	remaining: 2.1s
694:	learn: 0.3007074	total: 4.77s	remaining: 2.09s
695:	learn: 0.3005999	total: 4.77s	remaining: 2.08s
696:	learn: 0.3005995	total: 4.78s	remaining: 2.08s
697:	learn: 0.3005307	total: 4.78s	remaining: 2.07s
698:	learn: 0.3003942	total: 4.79s	remaining: 2.06s
699:	learn: 0.

842:	learn: 0.2807679	total: 5.88s	remaining: 1.09s
843:	learn: 0.2806439	total: 5.89s	remaining: 1.09s
844:	learn: 0.2805739	total: 5.9s	remaining: 1.08s
845:	learn: 0.2803854	total: 5.91s	remaining: 1.07s
846:	learn: 0.2803126	total: 5.91s	remaining: 1.07s
847:	learn: 0.2801800	total: 5.92s	remaining: 1.06s
848:	learn: 0.2801203	total: 5.92s	remaining: 1.05s
849:	learn: 0.2800215	total: 5.93s	remaining: 1.05s
850:	learn: 0.2799163	total: 5.94s	remaining: 1.04s
851:	learn: 0.2798026	total: 5.94s	remaining: 1.03s
852:	learn: 0.2797436	total: 5.95s	remaining: 1.02s
853:	learn: 0.2795341	total: 5.96s	remaining: 1.02s
854:	learn: 0.2791397	total: 5.96s	remaining: 1.01s
855:	learn: 0.2790235	total: 5.97s	remaining: 1s
856:	learn: 0.2790225	total: 5.97s	remaining: 996ms
857:	learn: 0.2786736	total: 5.98s	remaining: 989ms
858:	learn: 0.2785140	total: 5.98s	remaining: 982ms
859:	learn: 0.2782131	total: 5.99s	remaining: 975ms
860:	learn: 0.2781056	total: 6s	remaining: 968ms
861:	learn: 0.27801

0.8891891891891892

## GridSearchCV

In [47]:
param_grid = {'catboost__depth': [4, 7, 10],
          'catboost__learning_rate' : [0.03, 0.1, 0.15],
         'catboost__l2_leaf_reg': [1,4,9],
         'catboost__iterations': [300]}

grid_cat = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, scoring="roc_auc", verbose=1)
grid_cat.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
0:	learn: 0.6806821	total: 4.17ms	remaining: 1.25s
1:	learn: 0.6612839	total: 8.44ms	remaining: 1.26s
2:	learn: 0.6446522	total: 12.6ms	remaining: 1.25s
3:	learn: 0.6325154	total: 16.7ms	remaining: 1.24s
4:	learn: 0.6223327	total: 20.8ms	remaining: 1.23s
5:	learn: 0.6059856	total: 25.4ms	remaining: 1.24s
6:	learn: 0.5939457	total: 32.1ms	remaining: 1.34s
7:	learn: 0.5839443	total: 36.3ms	remaining: 1.32s
8:	learn: 0.5731705	total: 42.6ms	remaining: 1.38s
9:	learn: 0.5670308	total: 46.3ms	remaining: 1.34s
10:	learn: 0.5559151	total: 50.6ms	remaining: 1.33s
11:	learn: 0.5477570	total: 54.6ms	remaining: 1.31s
12:	learn: 0.5395084	total: 58.7ms	remaining: 1.3s
13:	learn: 0.5311941	total: 63.1ms	remaining: 1.29s
14:	learn: 0.5233399	total: 67.3ms	remaining: 1.28s
15:	learn: 0.5145897	total: 80.7ms	remaining: 1.43s
16:	learn: 0.5064778	total: 87ms	remaining: 1.45s
17:	learn: 0.5000933	total: 95.2ms	remaining: 1.49s
18:	learn: 0.495

188:	learn: 0.3387251	total: 868ms	remaining: 510ms
189:	learn: 0.3386321	total: 875ms	remaining: 507ms
190:	learn: 0.3383161	total: 882ms	remaining: 503ms
191:	learn: 0.3374839	total: 889ms	remaining: 500ms
192:	learn: 0.3373101	total: 896ms	remaining: 497ms
193:	learn: 0.3365758	total: 908ms	remaining: 496ms
194:	learn: 0.3360724	total: 912ms	remaining: 491ms
195:	learn: 0.3354998	total: 916ms	remaining: 486ms
196:	learn: 0.3352968	total: 920ms	remaining: 481ms
197:	learn: 0.3348538	total: 924ms	remaining: 476ms
198:	learn: 0.3347864	total: 929ms	remaining: 471ms
199:	learn: 0.3342457	total: 933ms	remaining: 466ms
200:	learn: 0.3332680	total: 937ms	remaining: 462ms
201:	learn: 0.3330550	total: 941ms	remaining: 457ms
202:	learn: 0.3327401	total: 945ms	remaining: 452ms
203:	learn: 0.3319238	total: 950ms	remaining: 447ms
204:	learn: 0.3314823	total: 954ms	remaining: 442ms
205:	learn: 0.3311107	total: 958ms	remaining: 437ms
206:	learn: 0.3305581	total: 962ms	remaining: 432ms
207:	learn: 

48:	learn: 0.4182817	total: 205ms	remaining: 1.05s
49:	learn: 0.4163550	total: 212ms	remaining: 1.06s
50:	learn: 0.4147739	total: 220ms	remaining: 1.07s
51:	learn: 0.4135656	total: 228ms	remaining: 1.09s
52:	learn: 0.4120563	total: 232ms	remaining: 1.08s
53:	learn: 0.4107226	total: 236ms	remaining: 1.08s
54:	learn: 0.4101351	total: 241ms	remaining: 1.07s
55:	learn: 0.4095153	total: 249ms	remaining: 1.08s
56:	learn: 0.4088927	total: 253ms	remaining: 1.08s
57:	learn: 0.4081834	total: 257ms	remaining: 1.07s
58:	learn: 0.4067909	total: 261ms	remaining: 1.07s
59:	learn: 0.4057300	total: 265ms	remaining: 1.06s
60:	learn: 0.4050954	total: 269ms	remaining: 1.05s
61:	learn: 0.4039276	total: 273ms	remaining: 1.05s
62:	learn: 0.4030579	total: 277ms	remaining: 1.04s
63:	learn: 0.4021926	total: 281ms	remaining: 1.03s
64:	learn: 0.4013736	total: 285ms	remaining: 1.03s
65:	learn: 0.4008556	total: 289ms	remaining: 1.02s
66:	learn: 0.4001134	total: 293ms	remaining: 1.02s
67:	learn: 0.3994164	total: 297

234:	learn: 0.3304547	total: 1s	remaining: 277ms
235:	learn: 0.3296829	total: 1.01s	remaining: 273ms
236:	learn: 0.3292618	total: 1.01s	remaining: 270ms
237:	learn: 0.3290423	total: 1.02s	remaining: 266ms
238:	learn: 0.3288044	total: 1.03s	remaining: 262ms
239:	learn: 0.3283302	total: 1.03s	remaining: 258ms
240:	learn: 0.3278688	total: 1.04s	remaining: 254ms
241:	learn: 0.3274140	total: 1.04s	remaining: 249ms
242:	learn: 0.3270545	total: 1.04s	remaining: 245ms
243:	learn: 0.3261598	total: 1.05s	remaining: 241ms
244:	learn: 0.3259553	total: 1.05s	remaining: 236ms
245:	learn: 0.3255058	total: 1.06s	remaining: 232ms
246:	learn: 0.3250155	total: 1.06s	remaining: 228ms
247:	learn: 0.3245882	total: 1.06s	remaining: 223ms
248:	learn: 0.3244965	total: 1.07s	remaining: 219ms
249:	learn: 0.3234336	total: 1.07s	remaining: 215ms
250:	learn: 0.3232758	total: 1.08s	remaining: 210ms
251:	learn: 0.3231415	total: 1.08s	remaining: 206ms
252:	learn: 0.3229047	total: 1.08s	remaining: 202ms
253:	learn: 0.3

96:	learn: 0.3897246	total: 404ms	remaining: 846ms
97:	learn: 0.3892433	total: 410ms	remaining: 846ms
98:	learn: 0.3882787	total: 417ms	remaining: 847ms
99:	learn: 0.3878887	total: 427ms	remaining: 854ms
100:	learn: 0.3876797	total: 431ms	remaining: 849ms
101:	learn: 0.3863031	total: 435ms	remaining: 845ms
102:	learn: 0.3858669	total: 439ms	remaining: 840ms
103:	learn: 0.3856603	total: 444ms	remaining: 836ms
104:	learn: 0.3851099	total: 448ms	remaining: 831ms
105:	learn: 0.3834416	total: 455ms	remaining: 832ms
106:	learn: 0.3832252	total: 459ms	remaining: 827ms
107:	learn: 0.3829978	total: 462ms	remaining: 822ms
108:	learn: 0.3827353	total: 468ms	remaining: 819ms
109:	learn: 0.3822599	total: 472ms	remaining: 815ms
110:	learn: 0.3816607	total: 476ms	remaining: 810ms
111:	learn: 0.3812042	total: 480ms	remaining: 805ms
112:	learn: 0.3807309	total: 484ms	remaining: 801ms
113:	learn: 0.3796932	total: 488ms	remaining: 796ms
114:	learn: 0.3787220	total: 492ms	remaining: 791ms
115:	learn: 0.37

...

283:	learn: 0.0936828	total: 17.4s	remaining: 979ms
284:	learn: 0.0933829	total: 17.4s	remaining: 918ms
285:	learn: 0.0932173	total: 17.5s	remaining: 857ms
286:	learn: 0.0930375	total: 17.6s	remaining: 796ms
287:	learn: 0.0928578	total: 17.6s	remaining: 735ms
288:	learn: 0.0926777	total: 17.7s	remaining: 674ms
289:	learn: 0.0924460	total: 17.8s	remaining: 613ms
290:	learn: 0.0922040	total: 17.8s	remaining: 552ms
291:	learn: 0.0919262	total: 17.9s	remaining: 491ms
292:	learn: 0.0917704	total: 18s	remaining: 430ms
293:	learn: 0.0915805	total: 18.1s	remaining: 368ms
294:	learn: 0.0914175	total: 18.1s	remaining: 307ms
295:	learn: 0.0912654	total: 18.2s	remaining: 246ms
296:	learn: 0.0910751	total: 18.3s	remaining: 184ms
297:	learn: 0.0907833	total: 18.3s	remaining: 123ms
298:	learn: 0.0906197	total: 18.4s	remaining: 61.5ms
299:	learn: 0.0904829	total: 18.5s	remaining: 0us


[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:  9.2min finished


0:	learn: 0.6744150	total: 9.65ms	remaining: 2.88s
1:	learn: 0.6563556	total: 42.4ms	remaining: 6.32s
2:	learn: 0.6394708	total: 58.9ms	remaining: 5.83s
3:	learn: 0.6260757	total: 64.6ms	remaining: 4.78s
4:	learn: 0.6126759	total: 69ms	remaining: 4.07s
5:	learn: 0.5999499	total: 74.7ms	remaining: 3.66s
6:	learn: 0.5887520	total: 78.9ms	remaining: 3.3s
7:	learn: 0.5781145	total: 151ms	remaining: 5.51s
8:	learn: 0.5684411	total: 161ms	remaining: 5.19s
9:	learn: 0.5580397	total: 165ms	remaining: 4.79s
10:	learn: 0.5483274	total: 175ms	remaining: 4.59s
11:	learn: 0.5374166	total: 182ms	remaining: 4.36s
12:	learn: 0.5307336	total: 187ms	remaining: 4.13s
13:	learn: 0.5241084	total: 191ms	remaining: 3.91s
14:	learn: 0.5200848	total: 195ms	remaining: 3.71s
15:	learn: 0.5122723	total: 205ms	remaining: 3.63s
16:	learn: 0.5052464	total: 225ms	remaining: 3.75s
17:	learn: 0.5001740	total: 231ms	remaining: 3.62s
18:	learn: 0.4953285	total: 238ms	remaining: 3.52s
19:	learn: 0.4898928	total: 243ms	rem

166:	learn: 0.3506153	total: 4.03s	remaining: 3.21s
167:	learn: 0.3505513	total: 4.04s	remaining: 3.18s
168:	learn: 0.3504168	total: 4.05s	remaining: 3.14s
169:	learn: 0.3500588	total: 4.14s	remaining: 3.16s
170:	learn: 0.3500518	total: 4.14s	remaining: 3.13s
171:	learn: 0.3494788	total: 4.22s	remaining: 3.14s
172:	learn: 0.3492330	total: 4.24s	remaining: 3.11s
173:	learn: 0.3486306	total: 4.32s	remaining: 3.13s
174:	learn: 0.3485245	total: 4.33s	remaining: 3.09s
175:	learn: 0.3481352	total: 4.35s	remaining: 3.06s
176:	learn: 0.3473775	total: 4.42s	remaining: 3.08s
177:	learn: 0.3466026	total: 4.45s	remaining: 3.05s
178:	learn: 0.3460194	total: 4.53s	remaining: 3.06s
179:	learn: 0.3459947	total: 4.53s	remaining: 3.02s
180:	learn: 0.3452721	total: 4.57s	remaining: 3s
181:	learn: 0.3446997	total: 4.65s	remaining: 3.01s
182:	learn: 0.3442834	total: 4.67s	remaining: 2.99s
183:	learn: 0.3442748	total: 4.68s	remaining: 2.95s
184:	learn: 0.3439803	total: 4.75s	remaining: 2.95s
185:	learn: 0.3

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('mapper', DataFrameMapper(default=False, df_out=True,
        features=[('Fare', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Fare'}), ('Pclass', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Pclass'}), ('Sex', Fill...lumn_names=['Age_cat'])), ('catboost', <catboost.core.CatBoostClassifier object at 0x7f3c4776cac8>)]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'catboost__depth': [4, 7, 10], 'catboost__learning_rate': [0.03, 0.1, 0.15], 'catboost__l2_leaf_reg': [1, 4, 9], 'catboost__iterations': [300]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=1)

In [48]:
grid_cat.best_score_

0.8605129837238501

## RandomizedSearchCV

In [49]:
param_grid = {'catboost__depth': [4, 7, 10],
          'catboost__learning_rate' : [0.03, 0.1, 0.15],
         'catboost__l2_leaf_reg': [1,4,9],
         'catboost__iterations': [300]}

grid_cat = RandomizedSearchCV(pipeline, cv=3, n_jobs=1, param_distributions=param_grid, scoring="roc_auc", verbose=1)
grid_cat.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
0:	learn: 0.6351726	total: 11.8ms	remaining: 3.52s
1:	learn: 0.5839659	total: 32.4ms	remaining: 4.83s
2:	learn: 0.5509055	total: 40.6ms	remaining: 4.02s
3:	learn: 0.5269782	total: 45.2ms	remaining: 3.35s
4:	learn: 0.4936832	total: 53.2ms	remaining: 3.14s
5:	learn: 0.4657003	total: 60ms	remaining: 2.94s
6:	learn: 0.4426813	total: 68.9ms	remaining: 2.88s
7:	learn: 0.4326286	total: 75.6ms	remaining: 2.76s
8:	learn: 0.4229587	total: 80.2ms	remaining: 2.59s
9:	learn: 0.4087543	total: 88.4ms	remaining: 2.56s
10:	learn: 0.4038676	total: 92.9ms	remaining: 2.44s
11:	learn: 0.3977831	total: 101ms	remaining: 2.42s
12:	learn: 0.3905005	total: 106ms	remaining: 2.35s
13:	learn: 0.3777505	total: 114ms	remaining: 2.34s
14:	learn: 0.3696822	total: 123ms	remaining: 2.33s
15:	learn: 0.3625258	total: 132ms	remaining: 2.35s
16:	learn: 0.3607384	total: 142ms	remaining: 2.37s
17:	learn: 0.3562426	total: 154ms	remaining: 2.41s
18:	learn: 0.3534757	t

178:	learn: 0.1262738	total: 1.56s	remaining: 1.05s
179:	learn: 0.1260254	total: 1.57s	remaining: 1.05s
180:	learn: 0.1251702	total: 1.58s	remaining: 1.04s
181:	learn: 0.1245016	total: 1.6s	remaining: 1.04s
182:	learn: 0.1239850	total: 1.61s	remaining: 1.03s
183:	learn: 0.1232021	total: 1.62s	remaining: 1.02s
184:	learn: 0.1221806	total: 1.63s	remaining: 1.01s
185:	learn: 0.1217008	total: 1.63s	remaining: 1s
186:	learn: 0.1215754	total: 1.64s	remaining: 992ms
187:	learn: 0.1212982	total: 1.65s	remaining: 983ms
188:	learn: 0.1210764	total: 1.66s	remaining: 974ms
189:	learn: 0.1206246	total: 1.67s	remaining: 965ms
190:	learn: 0.1202419	total: 1.67s	remaining: 955ms
191:	learn: 0.1199168	total: 1.68s	remaining: 946ms
192:	learn: 0.1195934	total: 1.69s	remaining: 937ms
193:	learn: 0.1189209	total: 1.7s	remaining: 928ms
194:	learn: 0.1185855	total: 1.71s	remaining: 919ms
195:	learn: 0.1181884	total: 1.71s	remaining: 910ms
196:	learn: 0.1177832	total: 1.72s	remaining: 901ms
197:	learn: 0.116

56:	learn: 0.2990727	total: 408ms	remaining: 1.74s
57:	learn: 0.2969795	total: 417ms	remaining: 1.74s
58:	learn: 0.2934046	total: 429ms	remaining: 1.75s
59:	learn: 0.2926754	total: 437ms	remaining: 1.75s
60:	learn: 0.2893480	total: 445ms	remaining: 1.74s
61:	learn: 0.2875555	total: 453ms	remaining: 1.74s
62:	learn: 0.2870249	total: 462ms	remaining: 1.74s
63:	learn: 0.2866392	total: 466ms	remaining: 1.72s
64:	learn: 0.2846886	total: 474ms	remaining: 1.71s
65:	learn: 0.2832586	total: 482ms	remaining: 1.71s
66:	learn: 0.2812182	total: 489ms	remaining: 1.7s
67:	learn: 0.2780625	total: 497ms	remaining: 1.7s
68:	learn: 0.2766039	total: 505ms	remaining: 1.69s
69:	learn: 0.2726975	total: 514ms	remaining: 1.69s
70:	learn: 0.2717131	total: 522ms	remaining: 1.68s
71:	learn: 0.2709596	total: 530ms	remaining: 1.68s
72:	learn: 0.2678195	total: 539ms	remaining: 1.68s
73:	learn: 0.2671182	total: 547ms	remaining: 1.67s
74:	learn: 0.2652557	total: 555ms	remaining: 1.67s
75:	learn: 0.2651963	total: 560ms

223:	learn: 0.1316735	total: 1.82s	remaining: 617ms
224:	learn: 0.1312600	total: 1.83s	remaining: 611ms
225:	learn: 0.1307143	total: 1.84s	remaining: 604ms
226:	learn: 0.1306541	total: 1.85s	remaining: 596ms
227:	learn: 0.1301396	total: 1.86s	remaining: 588ms
228:	learn: 0.1298140	total: 1.87s	remaining: 580ms
229:	learn: 0.1295927	total: 1.88s	remaining: 572ms
230:	learn: 0.1290269	total: 1.89s	remaining: 564ms
231:	learn: 0.1287126	total: 1.9s	remaining: 556ms
232:	learn: 0.1284816	total: 1.9s	remaining: 547ms
233:	learn: 0.1279965	total: 1.91s	remaining: 539ms
234:	learn: 0.1276479	total: 1.92s	remaining: 531ms
235:	learn: 0.1272698	total: 1.93s	remaining: 523ms
236:	learn: 0.1270801	total: 1.94s	remaining: 515ms
237:	learn: 0.1269934	total: 1.94s	remaining: 506ms
238:	learn: 0.1267236	total: 1.95s	remaining: 498ms
239:	learn: 0.1265904	total: 1.96s	remaining: 490ms
240:	learn: 0.1262609	total: 1.97s	remaining: 482ms
241:	learn: 0.1249017	total: 1.98s	remaining: 474ms
242:	learn: 0.

100:	learn: 0.2277239	total: 813ms	remaining: 1.6s
101:	learn: 0.2265269	total: 824ms	remaining: 1.6s
102:	learn: 0.2250936	total: 836ms	remaining: 1.6s
103:	learn: 0.2248355	total: 850ms	remaining: 1.6s
104:	learn: 0.2242556	total: 861ms	remaining: 1.6s
105:	learn: 0.2234247	total: 869ms	remaining: 1.59s
106:	learn: 0.2222562	total: 877ms	remaining: 1.58s
107:	learn: 0.2218887	total: 885ms	remaining: 1.57s
108:	learn: 0.2199296	total: 893ms	remaining: 1.56s
109:	learn: 0.2197324	total: 901ms	remaining: 1.56s
110:	learn: 0.2183974	total: 909ms	remaining: 1.55s
111:	learn: 0.2164461	total: 917ms	remaining: 1.54s
112:	learn: 0.2148914	total: 925ms	remaining: 1.53s
113:	learn: 0.2146517	total: 933ms	remaining: 1.52s
114:	learn: 0.2137935	total: 942ms	remaining: 1.51s
115:	learn: 0.2128807	total: 950ms	remaining: 1.51s
116:	learn: 0.2127550	total: 958ms	remaining: 1.5s
117:	learn: 0.2114450	total: 966ms	remaining: 1.49s
118:	learn: 0.2107021	total: 974ms	remaining: 1.48s
119:	learn: 0.2102

...

219:	learn: 0.1038606	total: 2.02s	remaining: 736ms
220:	learn: 0.1033354	total: 2.04s	remaining: 729ms
221:	learn: 0.1030059	total: 2.05s	remaining: 720ms
222:	learn: 0.1028165	total: 2.06s	remaining: 710ms
223:	learn: 0.1026850	total: 2.07s	remaining: 701ms
224:	learn: 0.1018648	total: 2.08s	remaining: 692ms
225:	learn: 0.1013121	total: 2.09s	remaining: 683ms
226:	learn: 0.1009157	total: 2.1s	remaining: 674ms
227:	learn: 0.1007623	total: 2.1s	remaining: 665ms
228:	learn: 0.1006280	total: 2.11s	remaining: 656ms
229:	learn: 0.1005708	total: 2.12s	remaining: 646ms
230:	learn: 0.1005214	total: 2.13s	remaining: 637ms
231:	learn: 0.1003753	total: 2.14s	remaining: 628ms
232:	learn: 0.0998722	total: 2.15s	remaining: 618ms
233:	learn: 0.0995920	total: 2.16s	remaining: 609ms
234:	learn: 0.0987633	total: 2.17s	remaining: 600ms
235:	learn: 0.0985831	total: 2.18s	remaining: 590ms
236:	learn: 0.0980512	total: 2.19s	remaining: 581ms
237:	learn: 0.0978201	total: 2.19s	remaining: 572ms
238:	learn: 0.

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  3.8min finished


0:	learn: 0.6760190	total: 4.83ms	remaining: 1.44s
1:	learn: 0.6596740	total: 9.51ms	remaining: 1.42s
2:	learn: 0.6428825	total: 18.9ms	remaining: 1.87s
3:	learn: 0.6271185	total: 26.6ms	remaining: 1.97s
4:	learn: 0.6142710	total: 36.2ms	remaining: 2.13s
5:	learn: 0.6008860	total: 42.3ms	remaining: 2.07s
6:	learn: 0.5881965	total: 51.9ms	remaining: 2.17s
7:	learn: 0.5799890	total: 56.8ms	remaining: 2.07s
8:	learn: 0.5736719	total: 61.4ms	remaining: 1.98s
9:	learn: 0.5667817	total: 66.2ms	remaining: 1.92s
10:	learn: 0.5568399	total: 71.1ms	remaining: 1.87s
11:	learn: 0.5481396	total: 76.1ms	remaining: 1.83s
12:	learn: 0.5409203	total: 82.1ms	remaining: 1.81s
13:	learn: 0.5334131	total: 91.5ms	remaining: 1.87s
14:	learn: 0.5267312	total: 99.4ms	remaining: 1.89s
15:	learn: 0.5200952	total: 104ms	remaining: 1.84s
16:	learn: 0.5148629	total: 108ms	remaining: 1.8s
17:	learn: 0.5072696	total: 117ms	remaining: 1.84s
18:	learn: 0.5007663	total: 127ms	remaining: 1.88s
19:	learn: 0.4958573	total:

161:	learn: 0.3650984	total: 1.22s	remaining: 1.04s
162:	learn: 0.3648565	total: 1.23s	remaining: 1.03s
163:	learn: 0.3642476	total: 1.24s	remaining: 1.03s
164:	learn: 0.3640229	total: 1.24s	remaining: 1.02s
165:	learn: 0.3638653	total: 1.25s	remaining: 1.01s
166:	learn: 0.3636311	total: 1.26s	remaining: 1s
167:	learn: 0.3635474	total: 1.26s	remaining: 993ms
168:	learn: 0.3632877	total: 1.27s	remaining: 986ms
169:	learn: 0.3630085	total: 1.28s	remaining: 980ms
170:	learn: 0.3627973	total: 1.29s	remaining: 973ms
171:	learn: 0.3624711	total: 1.3s	remaining: 967ms
172:	learn: 0.3622178	total: 1.31s	remaining: 960ms
173:	learn: 0.3616255	total: 1.32s	remaining: 953ms
174:	learn: 0.3614714	total: 1.32s	remaining: 946ms
175:	learn: 0.3607760	total: 1.33s	remaining: 937ms
176:	learn: 0.3605416	total: 1.34s	remaining: 929ms
177:	learn: 0.3603214	total: 1.34s	remaining: 920ms
178:	learn: 0.3599005	total: 1.35s	remaining: 914ms
179:	learn: 0.3596402	total: 1.36s	remaining: 907ms
180:	learn: 0.35

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('mapper', DataFrameMapper(default=False, df_out=True,
        features=[('Fare', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Fare'}), ('Pclass', [FillNa2(method='mean'), MinMaxScaler(copy=True, feature_range=(0, 1))], {'alias': 'Pclass'}), ('Sex', Fill...lumn_names=['Age_cat'])), ('catboost', <catboost.core.CatBoostClassifier object at 0x7f3c4776cac8>)]),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'catboost__depth': [4, 7, 10], 'catboost__learning_rate': [0.03, 0.1, 0.15], 'catboost__l2_leaf_reg': [1, 4, 9], 'catboost__iterations': [300]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='roc_auc', verbose=1)

In [50]:
grid_cat.best_score_

0.8592486509264752